## 环境设置
%conda create -n distilabel python=3.12.9
%conda activate distilabel

## 安装基础sdk包

In [ ]:
%pip install distilabel[ollama,openai]

## 先测试一下普通llm的应答效果

In [32]:
from distilabel.models.llms import OllamaLLM
from distilabel.models.llms import OpenAILLM

llm = OllamaLLM(model="llama3.2-vision:latest")
#llm = OllamaLLM(model="deepseek-r1:7b")
#llm = OpenAILLM(model="deepseek-r1:7b",base_url="http://127.0.0.1:11434/v1",api_key="akey")
llm.load()

# Call the model
output = llm.generate(inputs=[[{"role": "system", "content": "You are a helpful assistant,you always response in chinese"},{"role": "user", "content": "Hello world!"}]])
print(output[0]['generations'][0])

C:\Users\13802\AppData\Local\Temp\ipykernel_23860\786400136.py:4: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  llm = OllamaLLM(model="llama3.2-vision:latest")


你好，世界！


## deepseek 的响应

In [33]:
from distilabel.models.llms import OllamaLLM
from distilabel.models.llms import OpenAILLM

#llm = OllamaLLM(model="llama3.2-vision:latest")
llm = OllamaLLM(model="deepseek-r1:7b")
#llm = OpenAILLM(model="deepseek-r1:7b",base_url="http://127.0.0.1:11434/v1",api_key="akey")
llm.load()

# Call the model
output = llm.generate(inputs=[[{"role": "system", "content": "You are a helpful assistant,you always response in chinese"},{"role": "user", "content": "Hello world!"}]])
print(output)

extracted_content = output[0]['generations'][0].split('</think>\n\n')[1]
print(extracted_content)

C:\Users\13802\AppData\Local\Temp\ipykernel_23860\2556573109.py:5: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  llm = OllamaLLM(model="deepseek-r1:7b")


[{'generations': ["<think>\nOkay, so I need to figure out how to write the number 10 using exactly two numbers and only addition. Hmm, let's see... Well, if I think about it, adding two single-digit numbers can give me various results. For example, 5 plus 5 is 10, that seems straightforward. But wait, are there other combinations? Maybe like 6 plus 4 or 7 plus 3. Yeah, those also add up to 10. But the question didn't specify if the numbers had to be different or not, so technically, using two of the same number works too.\n\nI guess another way could be using more than two digits but only adding two numbers in total. Like 9 plus 1 is still 10, which fits the criteria. So there are multiple ways to do this, depending on what numbers you choose as long as their sum is 10. It's important that I use exactly two numbers and only addition, so subtraction or any other operations aren't allowed.\n\nAlso, considering if I can use zero somehow, but adding anything to zero won't help because it w

load dataset

In [ ]:
from distilabel.steps import LoadDataFromHub
import os
os.environ["ALL_PROXY"] = "http://127.0.0.1:1087"
loader = LoadDataFromHub(
    repo_id="Conard/fortune-telling",
    split="train")
loader.load()
(result,_)=next(loader.process())
for item in result:
    print(item['Question'])
    print(item['Response'])
    print(item['Complex_CoT'])


Step 'None' hasn't received a pipeline, and it hasn't been created within a `Pipeline` context. Please, use `with Pipeline() as pipeline:` and create the step within the context.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 loader.load()                                                                               │
│    8 (result,_)=next(loader.process())                                                           │
│    9 for item in result:                                                                         │
│ ❱ 10 │   print(item['question'])                                                                 │
│   11 │   print(item['response'])                                                                 │
│   12 │   print(item['Complex_CoT'])                                                              │
│   13                                                                                             │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │              exit = <IPython.core.autocall.ZMQExitAutocall object at 0x000001E59E04FE90>     │ │
│ │ extracted_content = '要写出数字10并使用恰好两个数，只能使用加法。可以考虑多种组合：\n\n- **… │ │
│ │                     + 5 = 10**：这是一个直接且简洁的解法。\n- **6 + 4 = 10*'+97              │ │
│ │       get_ipython = <bound method InteractiveShell.get_ipython of                            │ │
│ │                     <ipykernel.zmqshell.ZMQInteractiveShell object at 0x000001E59E292540>>   │ │
│ │                In = [                                                                        │ │
│ │                     │   '',                                                                  │ │
│ │                     │   "get_ipython().run_line_magic('pip', 'install distilabel[ollama]')", │ │
│ │                     │   'from distilabel.models.llms import OllamaLLM\n\nllm =               │ │
│ │                     OllamaLLM(model="llama3.2-vi'+199,                                       │ │
│ │                     │   'from distilabel.models.llms import OllamaLLM\n\nllm =               │ │
│ │                     OllamaLLM(model="llama3.2-vi'+182,                                       │ │
│ │                     │   'from distilabel.models.llms import OllamaLLM\n\nllm =               │ │
│ │                     OllamaLLM(model="llama3.2-vi'+182,                                       │ │
│ │                     │   "get_ipython().run_line_magic('pip', 'install                        │ │
│ │                     distilabel[ollama,openai]')",                                            │ │
│ │                     │   'output',                                                            │ │
│ │                     │   'from distilabel.models.llms import OllamaLLM\n\nllm =               │ │
│ │                     OllamaLLM(model="llama3.2-vi'+213,                                       │ │
│ │                     │   'output',                                                            │ │
│ │                     │   'from distilabel.models.llms import OllamaLLM\n\n#llm =              │ │
│ │                     OllamaLLM(model="llama3.2-v'+253,                                        │ │
│ │                     │   ... +39                                                              │ │
│ │                     ]                                                                        │ │
│ │              item = {                                                                        │ │
│ │                     │   'Question': '新房装修,大门对着电梯好不好?要如何化解?',               │ │
│ │                     │   'Response':                                                          │ │
│ │                     '根据传统风水学的观点，大门正对电梯易形成"开口煞"，电梯频繁升降会扰乱家… │ │
│ │                     玄关阻隔法\n在入门处设置L型屏风或文化砖玄关墙，高度以1'+2